In [ ]:
!pip install onnxruntime

import joblib
import onnx
import onnxruntime as ort
import pandas as pd
import numpy as np

# Hides all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Simulate data received from a request
data = {
    "Loan_ID": "LP001002",
    "Gender": "Male",
    "Married": "No",
    "Dependents": "0",
    "Education": "Graduate",
    "Self_Employed": "No",
    "ApplicantIncome": 5849,
    "CoapplicantIncome": 0.0,
    "LoanAmount": 120,
    "Loan_Amount_Term": 360.0,
    "Credit_History": 1.0,
    "Property_Area": "Urban",
}

In [ ]:
# Load prepocessor
scaler_path = "../models/preprocessor.save"
scaler = joblib.load(scaler_path)

In [ ]:
# Wrapper for all preprocessing steps
col_types = {'Gender':"str",
             'Married':"str",
             'Dependents':"str",
             'Education':"str",
             'Self_Employed':"str",
             'ApplicantIncome':"float",
             'CoapplicantIncome':"float",
             'LoanAmount':"float",
             'Loan_Amount_Term':"float",
             'Credit_History':"bool",
             'Property_Area':"str",
             'Loan_Status':"str"}

def transform(data):
    df = pd.DataFrame.from_dict(data, orient='index').transpose()
    
    # Drop unecessary variables
    if 'Loan_ID' in df.columns:
        df = df.drop(['Loan_ID'], axis=1)
    
    # Set variable types
    df = df.astype(dtype = {key: value for key, value in col_types.items() if key != 'Loan_Status'})
    df = df.replace("nan", None)
    
    # One-hot encoding of categorical variables
    df['Gender'] = df['Gender'].replace({'Female': False, 'Male': True})
    df['Married'] = df['Married'].replace({'No': False, 'Yes': True})
    df['Education'] = df['Education'].replace({'Not Graduate': False, 'Graduate': True})
    df['Self_Employed'] = df['Self_Employed'].replace({'No': False, 'Yes': True})
    df['Dependents_0'] = df['Dependents'].apply(lambda x: True if x == '0' else False)
    df['Dependents_1'] = df['Dependents'].apply(lambda x: True if x == '1' else False)
    df['Dependents_2'] = df['Dependents'].apply(lambda x: True if x == '2' else False)
    df['Dependents_3+'] = df['Dependents'].apply(lambda x: True if x == '3+' else False)
    df = df.drop('Dependents', axis=1)
    df['Property_Area_Rural'] = df['Property_Area'].apply(lambda x: True if x == 'Rural' else False)
    df['Property_Area_Semiurban'] = df['Property_Area'].apply(lambda x: True if x == 'Semiurban' else False)
    df['Property_Area_Urban'] = df['Property_Area'].apply(lambda x: True if x == 'Urban' else False)
    df = df.drop('Property_Area', axis=1)
    
    # Data normalization over data aet
    df = scaler.transform(df)

    return df.astype(np.float32)

In [ ]:
import requests
import json

In [ ]:
# Define the URL of the Flask service
url = "http://127.0.0.1:12345/predict"

In [ ]:
# Send the POST request
input_name = "input"
my_json = json.dumps({input_name: transform(data).tolist()})
response = requests.post(url, json=my_json)

In [ ]:
# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())